In [7]:
import pandas as pd

# Carregue os seus arquivos CSV
df_ghs = pd.read_csv('C:/Users/Usuario/Desktop/fds-harpie/GHS.csv')
df_frases_p = pd.read_csv('C:/Users/Usuario/Desktop/Fds-harpie/Frases p.csv')

# --- INÍCIO DA CORREÇÃO ---
# AJUSTE OS NOMES DAS COLUNAS AQUI CONFORME O SEU ARQUIVO GHS.csv
nome_coluna_classificacao = 'Classe de Perigo'
nome_coluna_categoria = 'Categoria'

# Criamos uma nova coluna que combina a classificação e a categoria.
# .astype(str) garante que a categoria seja tratada como texto antes de juntar.
df_ghs['chave_classificacao_completa'] = df_ghs[nome_coluna_classificacao].str.strip() + " " + df_ghs[nome_coluna_categoria].astype(str).str.strip()
# --- FIM DA CORREÇÃO ---

print("DataFrame GHS corrigido e pronto para uso!")
df_ghs.head() # Mostra as 5 primeiras linhas para vermos a nova coluna

DataFrame GHS corrigido e pronto para uso!


,ID_Perigo,Classe de Perigo,Categoria,Pictogramas,Palavra de Advertência,Código da Frase H,Texto da Frase H,Frases P (Prevenção),Frases P (Resposta),Frases P (Armazenamento),...,Risco_Geral,Via_Principal,Sintomas_Agudos,Sintomas_Tardios,Notas_Medico,Severidade_Incendio,Severidade_Derramamento,Severidade_Manuseio,Severidade_Reatividade,chave_classificacao_completa
0,Explo_Inst,Explosivos -,Instáveis,GHS01,PERIGO,H200:,Explosivo; instável,"P201, P250, P280",P370+P372+P380+P373,P401,...,3,NaN,NaN,NaN,NaN,5,5,5,5.0,Explosivos - Instáveis
1,Explo_D1_1,Explosivos -,Divisão 1.1,GHS01,PERIGO,H201:,Explosivo; perigo de explosão em massa,"P210, P230, P234, P240, P250, P280",P370+P372+P380+P373,P401,...,3,NaN,NaN,NaN,NaN,5,5,5,5.0,Explosivos - Divisão 1.1
2,Explo_D1_2,Explosivos -,Divisão 1.2,GHS01,PERIGO,H202:,Explosivo; perigo grave de projeção,"P210, P230, P234, P240, P250, P280",P370+P372+P380+P373,P401,...,3,NaN,NaN,NaN,NaN,5,5,5,5.0,Explosivos - Divisão 1.2
3,Explo_D1_3,Explosivos -,Divisão 1.3,GHS01,PERIGO,H203:,"Explosivo; perigo de incêndio, deslocamento de...","P210, P230, P234, P240, P250, P280",P370+P372+P380+P373,P401,...,3,NaN,NaN,NaN,NaN,5,5,5,5.0,Explosivos - Divisão 1.3
4,Explo_D1_4,Explosivos -,Divisão 1.4,GHS01,ATENÇÃO,H204:,Perigo de incêndio ou projeções,"P210, P234, P240, P250, P280","P370+P372+P380+P373, P370+P380+P375",P401,...,3,NaN,NaN,NaN,NaN,5,5,5,5.0,Explosivos - Divisão 1.4


In [20]:
def processar_classificacoes(lista_de_classificacoes: list):
    """
    Recebe uma lista de classificações de perigo e retorna um dicionário
    com todas as informações da Seção 2 da FDS.
    """
    if not lista_de_classificacoes:
        return None

    # 1. FILTRAR DADOS (usando a nossa nova coluna 'chave_classificacao_completa')
    dados_filtrados = df_ghs[df_ghs['chave_classificacao_completa'].isin(lista_de_classificacoes)]

    # Se não encontrar nenhuma correspondência, retorna um aviso
    if dados_filtrados.empty:
        print(f"Aviso: Nenhuma das classificações fornecidas foi encontrada: {lista_de_classificacoes}")
        return None

    # 2. PROCESSAR PALAVRA DE ADVERTÊNCIA
    palavra_advertencia = "N/A"
    if 'Perigo' in dados_filtrados['Palavra de Advertência'].unique():
        palavra_advertencia = 'Perigo'
    elif 'Atenção' in dados_filtrados['Palavra de Advertência'].unique():
        palavra_advertencia = 'Atenção'

    # 3. COLETAR PICTOGRAMAS
    pictogramas = dados_filtrados['Pictogramas'].dropna().unique().tolist()

    # 4. COLETAR FRASES DE PERIGO (H)
    frases_h = (dados_filtrados['Código da Frase H'] + " " + dados_filtrados['Texto da Frase H']).dropna().unique().tolist()

    # 5. COLETAR E PROCESSAR FRASES DE PRECAUÇÃO (P)
    codigos_p_coletados = []
    for coluna in ['Frases P (Prevenção)', 'Frases P (Resposta)', 'Frases P (Armazenamento)', 'Frases P (Armazenamento)', 'Frases P (Disposição)']:
        codigos_p_coletados.extend(dados_filtrados[coluna].dropna().tolist())

    codigos_p_unicos = list(set(codigos_p_coletados))
    frases_p_finais = df_frases_p[df_frases_p['Codigo_Prec'].isin(codigos_p_unicos)]['Texto_Prec'].tolist()

    # 6. MONTAR O RESULTADO FINAL
    resultado = {
        'palavra_advertencia': palavra_advertencia,
        'pictogramas': pictogramas,
        'frases_h': frases_h,
        'frases_p': frases_p_finais
    }

    return resultado

In [21]:
# --- CÉLULA DE TESTE ---

# Crie uma lista com uma ou mais classificações de perigo para testar
classificacoes_para_teste = [
    'Líquidos inflamáveis - Categoria 2',
    'Corrosão/irritação à pele - Categoria 2'
]

# Chama a função que criamos com a nossa lista de teste
info_fds = processar_classificacoes(classificacoes_para_teste)

# Imprime o resultado para vermos se está correto
import json
print(json.dumps(info_fds, indent=4, ensure_ascii=False))

{
    "palavra_advertencia": "N/A",
    "pictogramas": [
        "GHS02",
        "GHS07"
    ],
    "frases_h": [
        "H225:  Líquido e vapores altamente inflamáveis",
        "H315:  Provoca irritação à pele"
    ],
    "frases_p": [
        "Armazene em local bem ventilado. Mantenha em local fresco.",
        "Descarte o conteúdo/recipiente em conformidade com a legislação municipal/estadual/federal/internacional."
    ]
}
